In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

# <1> 모델 I/O

### 1. 프롬프트 생성
- PromptTemplate
- ChatPromptTemplate : 대화형, system prompt

In [ ]:
# 1) 프롬프트 생성
from langchain import PromptTemplate
template = "{product}를 홍보하기 위한 좋은 문구를 추천해줘"

prompt = PromptTemplate(
	input_variables=["product"], 
	template=template
)

prompt.format(product="카메라")

'카메라를 홍보하기 위한 좋은 문구를 추천해줘'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

chat_template.format_messages(name="Bob", user_input="What is your name?")

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]

### 2. LLM 

##### 2-1) openai

In [ ]:
## 2-1) openai
# OPENAI_API_KEY

from langchain.chat_models import ChatOpenAI
llm1 = ChatOpenAI(temperature=0, model_name="gpt-4o")
prompt = "진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"
print(llm1.predict(prompt)) # 왜 invoke가 아닌가

강아지입니다.


##### 2-2) huggingface - falcon

In [ ]:
from huggingface_hub import login
import os

hf_token = os.getenv("HUGGINGFACE_TOKEN")
login(token=hf_token)

In [ ]:
from langchain import HuggingFaceHub
model_name = "tiiuae/falcon-7b-instruct"
llm2 = HuggingFaceHub(repo_id = model_name)
completion = llm2(prompt)
print(completion)



### 3. 모델 성능 비교

In [ ]:
## 3) 모델 성능 비교
from langchain.model_laboratory import ModelLaboratory
model_lab = ModelLaboratory.from_llms([llm1, llm2])
model_lab.compare("대한민국의 가을은 몇 월부터 몇 월까지야?")

### 4. 파서

In [ ]:
# - **CommaSeparatedListOutputParser** : (파서) 콤마로 구분하여 결과 반환
# - **PydanticOuputParser** : (파서) 정의된 필드 타입에 맞게 자동 변환
# - **DatetimeOutputParser** : (파서) 날짜/시간 형태로 결과 반환
# - **XMLOutputParser** : (파서) XML 형태로 결과 반환

In [ ]:
# CommaSeparatedListOutputParser
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, max_tokens=2048, model_name='gpt-4o')

output_parser = CommaSeparatedListOutputParser() # 파서 초기화
format_instructions = output_parser.get_format_instructions() # 출력 형식 지정

prompt = PromptTemplate(
	template='7개의 팀을 보여줘 {subject}.\n{format_instructions}',
	input_variables=['subject'],
	partial_variables={'format_instructions':format_instructions}
	)
 
query = "한국의 야구팀은?"
output = llm.predict(text=prompt.format(subject=query))

parsed_result = output_parser.parse(output)
print(parsed_result)

['두산 베어스', 'LG 트윈스', '키움 히어로즈', 'SSG 랜더스', 'NC 다이노스', '롯데 자이언츠', 'KIA 타이거즈']


In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

parser = PydanticOutputParser(pydantic_object=Person)
format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    template="다음 사람 정보를 제공해 주세요:\n{format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": format_instructions}
)

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
output = llm.predict(text=prompt.format())
parsed = parser.parse(output)

print(parsed.name, parsed.age)


John Doe 30


In [ ]:
# DatetimeOutputParser
from langchain.output_parsers import DatetimeOutputParser

parser = DatetimeOutputParser()
format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    template="이번 달 첫 번째 월요일의 날짜를 알려줘.\n{format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": format_instructions}
)

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
output = llm.predict(text=prompt.format())
parsed = parser.parse(output)

print(parsed) 

2023-10-02 00:00:00


In [ ]:
# XMLOutputParser
from langchain.output_parsers import XMLOutputParser

parser = XMLOutputParser()
format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    template="XML 형식으로 고양이 정보(name, age)를 알려줘.\n{format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": format_instructions}
)

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
output = llm.predict(text=prompt.format())
parsed = parser.parse(output)

print(parsed)  # ➝ {'name': '나비', 'age': '3'}


{'cats': [{'cat': [{'name': 'Whiskers'}, {'age': '3'}]}, {'cat': [{'name': 'Mittens'}, {'age': '5'}]}, {'cat': [{'name': 'Shadow'}, {'age': '2'}]}]}


# <2> 데이터 연결

- ETL : 데이터를 한 곳에서 다른 한 곳으로 옮기는 과정
    - 추출 (Extract) : 여러 출처로 부터 데이터를 가져옴
    - 변환 (Transform) : 추출한 데이터를 분석하고 필요한 형태로 변환
    - 적재 (Load) : 변환된 데이터를 데이터 베이스나 웨어하우스에 저장

- 데이터 연결 구성 요소
    - 문서 가져오기 : 다양한 출처에서 문서를 가져오는 것. ETL에서 추출에 해당
    - 문서 변환 : 입력 데이터를 청크로 분할 하거나 다시 결합하는 작업. ETL에서 변환에 해당
    - 문서 임베딩 : 데이터를 벡터로 변환
    - 벡터 저장소 : 변환된 벡터를 저장/관리/검색. ETL에서 적재
    - 검색기 : 언어 모델과 결합할 관련 문서를 가져오기 위한 것으로 정보 검색을 위한 역할

- RAG 실습 필요 라이브러리
    - pypdf : pdf 파일 다루는 데 사용
    - tiktoken : 임베딩을 하는 데 사용
    - faiss-cpu : 벡터 유사도 검색을 위해 사용
    - sentence-transofmrers : 문장 또는 단락을 벡터로 변환하기 위해 사용

#### 1. 파일로드

In [ ]:

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./data/프롬프트엔지니어링매뉴얼.pdf")
document = loader.load()
document[:2]

Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 64 0 (offset 0)
Ignoring wrong pointing object 275 0 (offset 0)
Ignoring wrong pointing object 1583 0 (offset 0)
Ignoring wrong pointing object 1585 0 (offset 0)


[Document(metadata={'producer': 'iOS Version 18.3.2 (Build 22D82) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20250330064714Z00'00'", 'moddate': "D:20250330064714Z00'00'", 'source': './data/프롬프트엔지니어링매뉴얼.pdf', 'total_pages': 244, 'page': 0, 'page_label': '1'}, page_content='ঠ\n\u0cd0\x01\u0c5f\x01\n݂\n\u09b4\nߊ\n\x0f\x17\x11\x01\x0f\x15\x13\x11\x13\nҊ\u0a37\x01\x01 \u0a50\n\x01\x01\u07b9 \u0a50'),
 Document(metadata={'producer': 'iOS Version 18.3.2 (Build 22D82) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20250330064714Z00'00'", 'moddate': "D:20250330064714Z00'00'", 'source': './data/프롬프트엔지니어링매뉴얼.pdf', 'total_pages': 244, 'page': 1, 'page_label': '2'}, page_content='')]

#### 2. 임베딩 처리

##### 2-1) openai

In [ ]:

from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

## 2-1) openai 임베딩
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(document, embeddings)

text = "이 문장은 임베딩 될 문장입니다"
text_embedding = embeddings.embed_query(text)

##### 2-2) huggingface 임베딩./

In [ ]:
## 2-2) huggingface 임베딩
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all_MiniLM-L6-v2")
text_embedding = embeddings.embed_query(text)
print(text_embedding)

#### 3. 검색기 활용

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name='gpt-4o')

from langchain.chains import RetrievalQA
retriever = db.as_retriever()


qa = RetrievalQA.from_chain_type(
	llm = llm,
	chain_type="stuff",
	retriever =retriever)
	# chain_type
	# stuff : 모든 문서를 한 번에 붙여서 전달
	# map_reduce : 각각의 문서를 LLM에 넣어 부분적인 응답 생성 > 응답들을 LLM에 넣어 종합 요약 및 최종 답변
	# refine : 초기 답변 생성 후, 문서들을 하나씩 읽으면서 답변을 점진적으로 보완
	# map_rerank : 문서마다 개별적 응답 생성. score를 각각 부여하고 가장 적합한 응답 선택
	
query = "업무에 활용할 수 있는 프롬프트 방법은?"
result = qa({'query':query})
print(result)

{'query': '업무에 활용할 수 있는 프롬프트 방법은?', 'result': '업무에 활용할 수 있는 프롬프트 방법으로는 카드뉴스, 4컷 만화, 계획보고서, 인포그래픽, 분석보고서, 검토보고서 등을 위한 프롬프트 엔지니어링 매뉴얼이 있습니다. 각 업무 유형에 맞는 프롬프트를 설계하여 효과적으로 활용할 수 있습니다.'}


# <3> 체인

- 여러 구성 요소를 조합해서 하나의 파이프라인을 구성해주는 역할
    - LLMChain
    - SequentialChain
- LCEL : LLM, Prompt, Tool, Parser 등을 블록처럼 연결해서 하나의 파이프라인처럼 실행

##### 1. LLMChain

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name='gpt-4o')
template = "{country}의 수도는 어디야?"
prompt = PromptTemplate(template=template, input_variables=['country'])

chain = LLMChain(llm=llm, prompt=prompt)
chain.run('대한민국')

'대한민국의 수도는 서울입니다.'

##### 2. +SequentialChain

In [ ]:
# chain1,
prompt1 = PromptTemplate(
	input_variables = ['sentence'],
	template = "다음 문장을 한글로 번역하세요. {sentence}"
)
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="translation")

# chain2,
prompt2 = PromptTemplate.from_template(
	"다음 문장을 한 문장으로 요약하세요. {translation}"
)
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="summary")

# allchain
from langchain.chains import SequentialChain
all_chain = SequentialChain(
	chains=[chain1, chain2],
	input_variables=['sentence'],
	output_variables=['translation','summary']
)

sentence = '''Over the past decade, social media platforms have transformed the way people communicate, share ideas, and access news. While these platforms have enabled faster dissemination of information, they have also contributed to the spread of misinformation. In recent years, efforts have been made to regulate content, but the sheer volume of posts makes it difficult to monitor everything. As society becomes increasingly reliant on digital communication, the challenge of balancing freedom of expression with the need for accurate information continues to grow.'''
all_chain(sentence)

{'sentence': 'Over the past decade, social media platforms have transformed the way people communicate, share ideas, and access news. While these platforms have enabled faster dissemination of information, they have also contributed to the spread of misinformation. In recent years, efforts have been made to regulate content, but the sheer volume of posts makes it difficult to monitor everything. As society becomes increasingly reliant on digital communication, the challenge of balancing freedom of expression with the need for accurate information continues to grow.',
 'translation': '지난 10년 동안 소셜 미디어 플랫폼은 사람들이 소통하고, 아이디어를 공유하며, 뉴스를 접하는 방식을 변화시켰습니다. 이러한 플랫폼은 정보의 빠른 전파를 가능하게 했지만, 동시에 잘못된 정보의 확산에도 기여했습니다. 최근 몇 년 동안 콘텐츠를 규제하려는 노력이 이루어졌지만, 게시물의 방대한 양 때문에 모든 것을 감시하기는 어렵습니다. 사회가 디지털 소통에 점점 더 의존하게 됨에 따라 표현의 자유와 정확한 정보의 필요성 간의 균형을 맞추는 과제가 계속해서 커지고 있습니다.',
 'summary': '소셜 미디어 플랫폼은 소통 방식을 혁신하고 정보 전파를 가속화했지만, 잘못된 정보 확산 문제를 야기했으며, 표현의 자유와 정확한 정보 제공 간의 균형을 맞추는 것이 중요한 과제로 부각되고 있습니다.'}

##### 3. LCEL

In [ ]:
from langchain_core.runnables import RunnableMap, RunnablePassthrough
chain1 = prompt1 | llm
chain2 = prompt2 | llm

# 전체 체인: LCEL 방식 조합
full_chain = (
    {"sentence": RunnablePassthrough()}  # 입력 그대로 넘김
    | RunnableMap({"translation": chain1})  # 번역 결과 저장
    | RunnableMap({"summary": lambda x: chain2.invoke({"translation": x["translation"]})})  # 요약
)

result = full_chain.invoke({"sentence": sentence})
print(result)

{'summary': AIMessage(content='소셜 미디어 플랫폼은 소통 방식을 혁신했지만 잘못된 정보 확산 문제를 초래했으며, 표현의 자유와 정확한 정보 제공 간의 균형을 맞추는 것이 중요한 과제로 부각되고 있습니다.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 314, 'total_tokens': 368, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_92f14e8683', 'finish_reason': 'stop', 'logprobs': None}, id='run-b7124549-d4b8-4df6-aa8e-ad61ee73c209-0')}


In [ ]:
from langchain_core.runnables import RunnableMap

full_chain = (
    {"sentence": RunnablePassthrough()}
    | RunnableMap({
        "translation": chain1,
        "sentence": lambda x: x["sentence"]  # 원문 유지
    })
    | RunnableMap({
        "translation": lambda x: x["translation"],  # 번역 유지
        "summary": lambda x: chain2.invoke({"translation": x["translation"]})
    })
)

result = full_chain.invoke({"sentence": sentence})

print("번역 결과:", result["translation"])
print("요약 결과:", result["summary"])


번역 결과: content='지난 10년 동안 소셜 미디어 플랫폼은 사람들이 소통하고, 아이디어를 공유하며, 뉴스를 접하는 방식을 변화시켰습니다. 이러한 플랫폼은 정보의 빠른 전파를 가능하게 했지만, 동시에 잘못된 정보의 확산에도 기여했습니다. 최근 몇 년 동안 콘텐츠를 규제하려는 노력이 이루어졌지만, 게시물의 방대한 양 때문에 모든 것을 감시하기는 어렵습니다. 사회가 디지털 커뮤니케이션에 점점 더 의존하게 됨에 따라 표현의 자유와 정확한 정보의 필요성 사이의 균형을 맞추는 과제가 계속해서 커지고 있습니다.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 142, 'prompt_tokens': 116, 'total_tokens': 258, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_432e014d75', 'finish_reason': 'stop', 'logprobs': None} id='run-ce17bd76-ddb3-4463-9459-2933263fead8-0'
요약 결과: content='소셜 미디어 플랫폼은 소통 방식을 혁신했지만 잘못된 정보 확산 문제를 초래했으며, 디지털 커뮤니케이션 의존도가 높아짐에 따라 표현의 자유와 정확한 정보 제공 간의 균형을 맞추는 것이 중요해졌습니다.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 3

# <4> 메모리

- 대화 과정에서 발생하는 데이터를 저장
    - 모든 대화 유지
    - 최근 K개의 대화 유지
    - 대화를 요약해서 유지
- ConversationChain 사용

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0,
			model_name = 'gpt-4o'
			)
			
from langchain import ConversationChain
conversation = ConversationChain(llm=llm, verbose=True)


conversation.predict(input="진희는 강아지를 한마리 키우고 있습니다.")
conversation.predict(input="영수는 고양이를 두마리 키우고 있습니다.")
conversation.predict(input="진희와 영수가 키우는 동물은 총 몇마리?")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 진희는 강아지를 한마리 키우고 있습니다.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 진희는 강아지를 한마리 키우고 있습니다.
AI: 진희가 강아지를 키우고 있다니 정말 귀엽겠네요! 강아지의 이름은 무엇인가요? 그리고 어떤 종인지도 궁금해요. 강아지들은 각자 독특한 성격과 특성을 가지고 있어서 함께 지내는 시간이 정말 즐거울 것 같아요. 강아지와 함께하는 일상에 대해 더 이야기해 주시면 좋겠어요!
Human: 영수는 고양이를 두마리 키우고 있습니다.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatt

'진희는 강아지 한 마리를, 영수는 고양이 두 마리를 키우고 있으니, 총 세 마리의 동물을 키우고 있네요. 진희와 영수가 함께 동물들과 시간을 보내는 모습이 상상되네요. 동물들과의 일상은 항상 즐겁고 특별한 순간들로 가득할 것 같아요!'

In [ ]:
from langchain.memory import ConversationBufferMemory
#ConversationBufferMemory	전체 대화 이력 기억	완전한 대화 문맥 유지
#ConversationSummaryMemory	이전 대화를 요약하여 기억	메모리 절약, 장기 대화에 적합
#ConversationBufferWindowMemory	최근 n개만 기억	최신 대화에만 집중
#VectorStoreRetrieverMemory	벡터 DB 기반 유사도 기억	복잡한 문맥 대응 가능 (RAG 등

memory = ConversationBufferMemory(llm=llm)

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)
conversation.predict(input = '안녕')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 안녕
AI:

> Finished chain.


'안녕하세요! 만나서 반가워요. 저는 AI입니다. 어떻게 도와드릴까요?'

In [ ]:
conversation.predict(input = '우리집 고양이 이름은 아쿠랑 마린이야')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕
AI: 안녕하세요! 만나서 반가워요. 저는 AI입니다. 어떻게 도와드릴까요?
Human: 우리집 고양이 이름은 아쿠랑 마린이야
AI:

> Finished chain.


'아쿠와 마린이라니, 정말 귀여운 이름이네요! 고양이들은 어떤 성격을 가지고 있나요? 아쿠는 장난꾸러기인가요, 아니면 마린이 더 활발한가요? 고양이들과 함께하는 일상 이야기를 듣고 싶어요!'

In [ ]:
conversation.predict(input = '우리집 고양이 이름이 뭐라고?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕
AI: 안녕하세요! 만나서 반가워요. 저는 AI입니다. 어떻게 도와드릴까요?
Human: 우리집 고양이 이름은 아쿠랑 마린이야
AI: 아쿠와 마린이라니, 정말 귀여운 이름이네요! 고양이들은 어떤 성격을 가지고 있나요? 아쿠는 장난꾸러기인가요, 아니면 마린이 더 활발한가요? 고양이들과 함께하는 일상 이야기를 듣고 싶어요!
Human: 우리집 고양이 이름이 뭐라고?
AI:

> Finished chain.


'당신의 고양이 이름은 아쿠와 마린이라고 하셨죠! 정말 사랑스러운 이름이에요. 아쿠와 마린이 어떤 특별한 행동이나 습관이 있는지 궁금하네요. 고양이들과 함께하는 일상 이야기를 더 들려주시면 좋겠어요!'

# <5> 에이전트/툴

In [ ]:
# 에드시런 생년월일을 위키피디아에서 조회 > 계산기를 이용해 나이 계산
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name='gpt-4o')

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(["wikipedia", "llm-math"], llm=llm)
# Tool Type
# python_repl : 파이썬 코드 실행 환경
# searx-serach : 오픈 소스 검색 엔진 serx 사용
# arxiv : arXiv 논문 검색 등
agent = initialize_agent(tools, # 에이전트가 접근할 수 있는 툴
		llm,
		agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, # 툴의 용도와 사용 시기를 결정하는 에이전트. 툴마다 설명 제공
		description='계산이 필요할 때', 
		verbose=True)
agent.run("에드 시런이 태어난 해는? 2024년도 현재 에드  시런은 몇 살?")

# AgentType
# REACT_DOCSTORE : 질문에 답하기 위해 관련 정보 조회
# CONVERSATIONAL_REACT_DESCRIPTION : 메모리를 사용하여 과거에 시도했던 대화 기억
# SELF_ASK_WITH_SEARCH : 추가 질문을 스스로 만든 뒤, 검색 툴로 답을 찾아 종합
# OPENAI_MULTI_FUNCTIONS : funcion-calling 확장형. 복수 선택

C:\Users\haesa\AppData\Local\Temp\ipykernel_30048\242275622.py:14: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, # 에이전트가 접근할 수 있는 툴




> Entering new AgentExecutor chain...
에드 시런이 태어난 해를 알아야 합니다. 이를 위해 Wikipedia를 사용하겠습니다.  
Action: wikipedia  
Action Input: 에드 시런  

Observation: Page: Rolling in the Deep
Summary: "Rolling in the Deep" is a song by English singer-songwriter Adele from her second studio album, 21 (2011). It is the lead single and opening track on the album. The song was written by Adele and Paul Epworth. The singer herself describes it as a "dark blues-y gospel disco tune". In 2011, it was reportedly the biggest crossover hit in the United States since 1985; "Rolling in the Deep" gained radio airplay from many different radio formats. It was released on 29 November 2010 as the lead single from 21 in digital download format. The lyrics describe the emotions of a scorned lover. The song's official music video uploaded to YouTube on 30 November 2010 has accumulated over 2.58 billion views and 13 million "Likes" as of 19 October 2024, similar to that of Adele's another hit "Someone like You" released 10 m

'에드 시런은 1991년에 태어났으며, 2024년에는 33살입니다.'

In [ ]:
tools = load_tools(["wikipedia", "llm-math", "arxiv"], llm=llm)
# Tool Type
# python_repl : 파이썬 코드 실행 환경
# searx-serach : 오픈 소스 검색 엔진 serx 사용
# arxiv : arXiv 논문 검색 등
agent = initialize_agent(tools, # 에이전트가 접근할 수 있는 툴
		llm,
		agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, # 툴의 용도와 사용 시기를 결정하는 에이전트. 툴마다 설명 제공
		description='아무때나', 
		verbose=True)
agent.run("What's the paper 1605.08386 about?")



> Entering new AgentExecutor chain...
To find out what the paper with the identifier 1605.08386 is about, I should search for it on arXiv. 

Action: arxiv
Action Input: 1605.08386
Observation: Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allowed moves of arbitrary length. We show that the diameter of these graphs on
fibers of a fixed integer matrix can be bounded from above by a constant. We
then study the mixing behaviour of heat-bath random walks on these graphs. We
also state explicit conditions on the set of moves so that the heat-bath random
walk, a generalization of the Glauber dynamics, is an expander in fixed
dimension.
Thought:The paper titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch studies graphs on lattice points where edges are defined by a finite set of allowed moves of arbitrary l

'The paper "Heat-bath random walks with Markov bases" studies graphs on lattice points with edges from a finite set of moves, showing bounded diameter and exploring heat-bath random walks\' mixing behavior, providing conditions for the walk to be an expander.'